In [1]:
import os
import sys
import csv
import pandas as pd
from pathlib import Path
from urllib.request import urlretrieve

In [2]:
def print_header(text):
    """헤더 출력"""
    print("\n" + "=" * 70)
    print(f"  {text}")
    print("=" * 70)


def print_step(num, text):
    """단계 출력"""
    print(f"\n[단계 {num}] {text}")
    print("-" * 70)


def check_dependencies():
    """필수 라이브러리 확인"""
    print_step(1, "필수 라이브러리 확인")
    
    required_packages = {
        'torch': 'PyTorch',
        'pandas': 'Pandas',
        'numpy': 'NumPy',
    }
    
    optional_packages = {
        'sentencepiece': 'SentencePiece',
        'matplotlib': 'Matplotlib',
    }
    
    missing_required = []
    missing_optional = []
    
    # 필수 패키지 확인
    for package, name in required_packages.items():
        try:
            __import__(package)
            print(f"✓ {name} 설치됨")
        except ImportError:
            print(f"❌ {name} 미설치")
            missing_required.append(package)
    
    # 선택 패키지 확인
    for package, name in optional_packages.items():
        try:
            __import__(package)
            print(f"✓ {name} 설치됨 (선택)")
        except ImportError:
            print(f"⚠ {name} 미설치 (선택)")
            missing_optional.append(package)
    
    if missing_required:
        print(f"\n❌ 필수 패키지가 설치되지 않았습니다:")
        for pkg in missing_required:
            print(f"   pip install {pkg}")
        return False
    
    if missing_optional:
        print(f"\n⚠ 선택 패키지가 설치되지 않았습니다:")
        for pkg in missing_optional:
            print(f"   pip install {pkg}")
        print("   (계속 진행할 수 있지만 일부 기능이 제한될 수 있습니다)")
    
    print("\n✓ 필수 라이브러리 모두 설치됨")
    return True

In [3]:
def download_dataset(force=False):
    """데이터셋 다운로드"""
    print_step(2, "ChatbotData.csv 다운로드")
    
    dataset_path = Path("ChatbotData.csv")
    
    if dataset_path.exists() and not force:
        print(f"✓ 데이터셋 이미 존재: {dataset_path.absolute()}")
        
        # 파일 크기 확인
        size_mb = dataset_path.stat().st_size / (1024 * 1024)
        print(f"  파일 크기: {size_mb:.2f} MB")
        
        # 데이터 샘플 확인
        try:
            df = pd.read_csv(dataset_path, encoding='utf-8', nrows=5)
            print(f"  컬럼: {list(df.columns)}")
            print(f"  샘플 개수: (파일 확인 중...)")
            
            df_full = pd.read_csv(dataset_path, encoding='utf-8')
            print(f"  전체 행: {len(df_full)}")
            print(f"\n  샘플 데이터:")
            for idx, row in df.iterrows():
                if idx < 2:
                    q = row[df.columns[0]][:30]
                    a = row[df.columns[1]][:30]
                    print(f"    Q: {q}...")
                    print(f"    A: {a}...")
        except Exception as e:
            print(f"  ⚠ 파일 읽기 실패: {e}")
        
        return True
    
    print("데이터셋 다운로드 중...")
    url = "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv"
    
    try:
        def progress_hook(block_num, block_size, total_size):
            downloaded = block_num * block_size
            percent = min(100, int(100 * downloaded / total_size))
            print(f"  진행률: {percent}% ({downloaded/1024/1024:.1f}MB / {total_size/1024/1024:.1f}MB)", end='\r')
        
        urlretrieve(url, dataset_path, progress_hook)
        print("\n✓ 데이터셋 다운로드 완료")
        
        # 파일 검증
        if dataset_path.exists():
            size_mb = dataset_path.stat().st_size / (1024 * 1024)
            print(f"✓ 파일 저장: {dataset_path.absolute()}")
            print(f"✓ 파일 크기: {size_mb:.2f} MB")
            
            # 데이터 확인
            try:
                df = pd.read_csv(dataset_path, encoding='utf-8')
                print(f"✓ 데이터 행: {len(df)}")
                print(f"✓ 컬럼: {list(df.columns)}")
            except Exception as e:
                print(f"❌ 데이터 읽기 실패: {e}")
                return False
            
            return True
        else:
            print("❌ 다운로드 실패")
            return False
    
    except Exception as e:
        print(f"\n❌ 다운로드 중 오류: {e}")
        print("\n수동 다운로드 방법:")
        print("1. 다음 URL로 이동:")
        print(f"   {url}")
        print("2. 파일 다운로드")
        print("3. 현재 디렉토리에 저장:")
        print(f"   {dataset_path.absolute()}")
        return False


In [4]:
def setup_directories():
    """디렉토리 설정"""
    print_step(3, "디렉토리 설정")
    
    checkpoints_dir = Path("checkpoints")
    checkpoints_dir.mkdir(exist_ok=True)
    
    print(f"✓ checkpoints 디렉토리: {checkpoints_dir.absolute()}")



In [5]:
def create_config_file():
    """설정 파일 생성 (선택)"""
    print_step(4, "설정 파일 생성")
    
    config_suggestions = {
        '1': ('CPU 최적화', {
            'D_MODEL': 256,
            'NUM_LAYERS': 2,
            'UNITS': 1024,
            'BATCH_SIZE': 16,
            'NUM_EPOCHS': 20
        }),
        '2': ('GPU 표준', {
            'D_MODEL': 512,
            'NUM_LAYERS': 4,
            'UNITS': 2048,
            'BATCH_SIZE': 32,
            'NUM_EPOCHS': 40
        }),
        '3': ('GPU 고성능', {
            'D_MODEL': 768,
            'NUM_LAYERS': 6,
            'UNITS': 4096,
            'BATCH_SIZE': 64,
            'NUM_EPOCHS': 60
        }),
    }
    
    print("권장 설정:")
    for key, (name, config) in config_suggestions.items():
        print(f"  {key}. {name}")
        print(f"     D_MODEL={config['D_MODEL']}, "
              f"BATCH_SIZE={config['BATCH_SIZE']}, "
              f"EPOCHS={config['NUM_EPOCHS']}")
    
    print("\n→ Config 클래스를 직접 수정하거나")
    print("→ korean_chatbot_complete.py의 설정값을 변경하세요")



In [6]:
def show_requirements():
    """시스템 요구사항 표시"""
    print_step(5, "시스템 요구사항")
    
    print("""
권장 사양:
├─ Python: 3.8 이상
├─ RAM: 8GB 이상
├─ 저장공간: 1GB 이상
└─ GPU (선택): NVIDIA CUDA 11.0+

GPU 없을 경우:
- CPU에서 작동 (느림)
- D_MODEL=256, NUM_LAYERS=2로 설정
- BATCH_SIZE=8로 설정
- 학습 시간: 24~48시간

GPU 있을 경우:
- D_MODEL=512, NUM_LAYERS=4로 설정
- BATCH_SIZE=32 이상으로 설정
- 학습 시간: 2~4시간

현재 시스템:
""")
    
    try:
        import torch
        print(f"  Python: {sys.version.split()[0]}")
        print(f"  PyTorch: {torch.__version__}")
        print(f"  CUDA: {'사용 가능' if torch.cuda.is_available() else '사용 불가'}")
        if torch.cuda.is_available():
            print(f"  GPU: {torch.cuda.get_device_name(0)}")
            print(f"  GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
    except Exception as e:
        print(f"  정보 조회 실패: {e}")



In [8]:
def main():
    """메인 함수"""
    print_header("한국어 Transformer 챗봇 - 빠른 시작 설정")
    
    print("""
이 스크립트는 다음을 수행합니다:
1. 필수 라이브러리 확인
2. ChatbotData.csv 다운로드
3. 디렉토리 설정
4. 시스템 정보 표시
5. 다음 단계 안내
""")
    
    # 1. 라이브러리 확인
    if not check_dependencies():
        print("\n❌ 필수 라이브러리를 설치하세요:")
        print("   pip install torch pandas numpy")
        print("   pip install sentencepiece matplotlib")
        sys.exit(1)
    
    # 2. 데이터 다운로드
    if not download_dataset():
        print("\n⚠ 데이터셋을 수동으로 다운로드 후 진행하세요.")
        response = input("계속 진행하시겠습니까? (y/n): ")
        if response.lower() != 'y':
            sys.exit(1)
    
    # 3. 디렉토리 설정
    setup_directories()
    
    # 4. 설정 제안
    create_config_file()
    
    # 5. 요구사항 표시
    show_requirements()
    
    # 6. 다음 단계
    show_next_steps()
    
    print_header("설정 완료!")
    
    print("""
이제 다음 명령으로 학습을 시작하세요:

   python korean_chatbot_complete.py

더 자세한 사용법은 USAGE_GUIDE_KO.md를 참고하세요.
""")


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\n설정이 중단되었습니다.")
        sys.exit(0)
    except Exception as e:
        print(f"\n❌ 오류 발생: {e}")
        sys.exit(1)



  한국어 Transformer 챗봇 - 빠른 시작 설정

이 스크립트는 다음을 수행합니다:
1. 필수 라이브러리 확인
2. ChatbotData.csv 다운로드
3. 디렉토리 설정
4. 시스템 정보 표시
5. 다음 단계 안내


[단계 1] 필수 라이브러리 확인
----------------------------------------------------------------------
✓ PyTorch 설치됨
✓ Pandas 설치됨
✓ NumPy 설치됨
✓ SentencePiece 설치됨 (선택)
✓ Matplotlib 설치됨 (선택)

✓ 필수 라이브러리 모두 설치됨

[단계 2] ChatbotData.csv 다운로드
----------------------------------------------------------------------
데이터셋 다운로드 중...
  진행률: 100% (0.9MB / 0.8MB)
✓ 데이터셋 다운로드 완료
✓ 파일 저장: /home/jovyan/work/ChatbotData.csv
✓ 파일 크기: 0.85 MB
✓ 데이터 행: 11823
✓ 컬럼: ['Q', 'A', 'label']

[단계 3] 디렉토리 설정
----------------------------------------------------------------------
✓ checkpoints 디렉토리: /home/jovyan/work/checkpoints

[단계 4] 설정 파일 생성
----------------------------------------------------------------------
권장 설정:
  1. CPU 최적화
     D_MODEL=256, BATCH_SIZE=16, EPOCHS=20
  2. GPU 표준
     D_MODEL=512, BATCH_SIZE=32, EPOCHS=40
  3. GPU 고성능
     D_MODEL=768, BATCH_SIZE=64, EPOCHS=60

→ Config 클래스를 직접

In [3]:
import os
import re
import csv
import math
import json
import warnings
from pathlib import Path
from datetime import datetime
from collections import defaultdict
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

try:
    import sentencepiece as spm
except ImportError:
    print("sentencepiece 설치 필요: pip install sentencepiece")

warnings.filterwarnings('ignore')


In [4]:
# ============================================================================
# 1. 설정 클래스
# ============================================================================

class Config:
    """모든 설정을 관리하는 클래스"""
    
    # 프로젝트 설정
    PROJECT_NAME = "korean_transformer_chatbot_v2"
    SAVE_DIR = Path("checkpoints")
    SAVE_DIR.mkdir(exist_ok=True)
    
    # ========== 데이터 설정 ==========
    DATA_PATH = "korean_qa_data.csv"  # 다운로드 받은 파일 경로
    VOCAB_SIZE_SPM = 8000          # SentencePiece 어휘 크기
    MAX_SEQ_LENGTH = 50            # 최대 시퀀스 길이
    MAX_SAMPLES = None             # None = 전체, 정수 = 제한
    TRAIN_VALID_SPLIT = 0.9        # 학습:검증 비율
    
    # ========== 모델 아키텍처 (개선됨) ==========
    D_MODEL = 768                  # 임베딩 차원
    NUM_LAYERS = 6                 # 인코더/디코더 레이어
    NUM_HEADS = 8                  # 멀티헤드 어텐션 헤드 수
    UNITS = 3072                   # 피드포워드 네트워크 차원
    DROPOUT = 0.3                  # 드롭아웃 비율
    
    # ========== 학습 설정 ==========
    BATCH_SIZE = 16                # 배치 크기 (GPU 메모리 고려)
    NUM_EPOCHS = 100                # 에폭 수
    LEARNING_RATE = 0.0001         # 기본 학습률
    WARMUP_STEPS = 8000            # 워밍업 스텝
    ADAM_BETAS = (0.9, 0.98)       # Adam 베타값
    WEIGHT_DECAY = 1e-4            # L2 정규화
    GRADIENT_CLIP = 0.5            # 그래디언트 클리핑
    LABEL_SMOOTHING = 0.2          # 라벨 스무딩
    
    # ========== 정규화 및 최적화 ==========
    USE_MIXED_PRECISION = True     # 혼합 정밀도 학습
    EARLY_STOPPING_PATIENCE = 15    # 조기 종료 인내도
    SAVE_INTERVAL = 500            # 체크포인트 저장 간격
    LOG_INTERVAL = 100             # 로그 출력 간격
    
    # ========== 추론 설정 ==========
    BEAM_SIZE = 5                  # Beam search 빔 크기
    LENGTH_PENALTY = 0.6           # 길이 패널티
    TEMPERATURE = 0.8              # 온도
    TOP_K = 40                     # Top-k 샘플링
    TOP_P = 0.9                    # Nucleus 샘플링
    
    # ========== 기기 설정 ==========
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    SEED = 42
    
    @classmethod
    def to_dict(cls):
        """설정을 JSON 직렬화 가능한 딕셔너리로 변환"""
        config_dict = {}
        for k, v in cls.__dict__.items():
            if not k.startswith('_') and k.isupper():
                # Path 객체를 문자열로 변환
                if isinstance(v, Path):
                    config_dict[k] = str(v)
                # torch.device 객체를 문자열로 변환
                elif hasattr(torch, 'device') and isinstance(v, torch.device):
                    config_dict[k] = str(v)
                # 튜플은 리스트로 변환 (JSON 호환성)
                elif isinstance(v, tuple):
                    config_dict[k] = list(v)
                # 기본 타입은 그대로
                elif isinstance(v, (str, int, float, bool, type(None))):
                    config_dict[k] = v
                # 나머지는 문자열로 변환
                else:
                    config_dict[k] = str(v)
        return config_dict
    
    @classmethod
    def save(cls, path: str = "config.json"):
        """설정을 JSON 파일로 저장"""
        try:
            config_dict = cls.to_dict()
            with open(path, 'w', encoding='utf-8') as f:
                json.dump(config_dict, f, ensure_ascii=False, indent=2)
            print(f"✓ 설정 저장: {path}")
            return True
        except Exception as e:
            print(f"⚠ 설정 저장 실패: {e}")
            # 저장 실패해도 프로그램은 계속 진행
            return False
    
    @classmethod
    def load(cls, path: str = "config.json") -> dict:
        """설정을 JSON 파일로부터 로드"""
        try:
            if Path(path).exists():
                with open(path, 'r', encoding='utf-8') as f:
                    return json.load(f)
            else:
                print(f"⚠ 설정 파일 없음: {path}")
                return {}
        except Exception as e:
            print(f"⚠ 설정 로드 실패: {e}")
            return {}



In [5]:
# ============================================================================
# 2. 한국어 전처리 함수
# ============================================================================

class KoreanPreprocessor:
    """한국어 텍스트 전처리"""
    
    # 특수 토큰 정의
    SPECIAL_TOKENS = {
        '<BOS>': '[BOS]',
        '<EOS>': '[EOS]',
        '<PAD>': '[PAD]',
        '<UNK>': '[UNK]',
    }
    
    @staticmethod
    def clean_text(text: str) -> str:
        """한국어 텍스트 정제"""
        if not isinstance(text, str):
            return ""
        
        # 공백 정규화
        text = text.strip()
        text = re.sub(r'\s+', ' ', text)
        
        # 한글, 영문, 숫자, 기본 구두점만 유지
        # ㄱ-ㅣ (한글 자모), ㄀-ㅟ (완성 한글), ㄰-ㅲ 포함
        text = re.sub(
            r'[^\w\s\.\!\?\,\;\:\(\)\-\'\"\u3130-\u318F\uAC00-\uD7AF]',
            '',
            text,
            flags=re.UNICODE
        )
        
        # 연속된 특수문자 정리
        text = re.sub(r'[\.\!\?\,\;]+', lambda m: m.group(0)[0] + ' ', text)
        
        # 다시 공백 정규화
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    
    @staticmethod
    def preprocess_sentence(text: str) -> str:
        """문장 전처리"""
        text = KoreanPreprocessor.clean_text(text)
        text = text.lower()  # 소문자 변환 (영문 부분)
        
        # 문장 끝에 마침표 추가 (없으면)
        if text and text[-1] not in '.!?,;:':
            text = text + '.'
        
        return text
    
    @staticmethod
    def load_data_from_csv(filepath: str, max_samples: int = None) -> List[Tuple[str, str]]:
        """
        ChatbotData.csv 형식으로부터 데이터 로드
        CSV 형식: Q (질문), A (답변), label (의도) 등
        """
        pairs = []
        
        try:
            df = pd.read_csv(filepath, encoding='utf-8')
            
            # CSV 컬럼명 확인
            print(f"✓ 데이터 컬럼: {df.columns.tolist()}")
            
            # Q, A 또는 question, answer 컬럼 찾기
            q_col = None
            a_col = None
            
            for col in df.columns:
                if col.lower() in ['q', 'question']:
                    q_col = col
                elif col.lower() in ['a', 'answer']:
                    a_col = col
            
            if q_col is None or a_col is None:
                # 첫 두 컬럼 사용
                q_col = df.columns[0]
                a_col = df.columns[1]
                print(f"⚠ Q/A 컬럼 자동 선택: {q_col}, {a_col}")
            
            # 데이터 로드
            for idx, row in df.iterrows():
                if max_samples and len(pairs) >= max_samples:
                    break
                
                q = str(row[q_col]).strip()
                a = str(row[a_col]).strip()
                
                # 유효성 검사
                if q and a and len(q) > 1 and len(a) > 1:
                    q_clean = KoreanPreprocessor.preprocess_sentence(q)
                    a_clean = KoreanPreprocessor.preprocess_sentence(a)
                    
                    if q_clean and a_clean:
                        pairs.append((q_clean, a_clean))
            
            print(f"✓ {len(pairs)}개 Q&A 쌍 로드됨")
            return pairs
            
        except FileNotFoundError:
            print(f"❌ 파일 없음: {filepath}")
            print("파일을 다음 경로에 놓아주세요:")
            print(f"  {Path(filepath).absolute()}")
            return []
        except Exception as e:
            print(f"❌ 데이터 로드 오류: {e}")
            return []



In [6]:
# ============================================================================
# 3. SentencePiece 토크나이저 설정
# ============================================================================

def train_tokenizer(pairs: List[Tuple[str, str]], config: Config):
    """SentencePiece 토크나이저 학습"""
    
    corpus_file = "corpus.txt"
    
    # 코퍼스 저장
    with open(corpus_file, 'w', encoding='utf-8') as f:
        for q, a in pairs:
            f.write(q + '\n')
            f.write(a + '\n')
    
    print(f"✓ 코퍼스 저장: {corpus_file}")
    
    # SentencePiece 학습
    spm.SentencePieceTrainer.Train(
        input=corpus_file,
        model_prefix="spm_korean",
        vocab_size=config.VOCAB_SIZE_SPM,
        character_coverage=1.0,
        model_type="bpe",
        max_sentence_length=999999,
        bos_id=1,
        eos_id=2,
        pad_id=0,
        unk_id=3,
        user_defined_symbols=['[SEP]'],
        normalization_rule_name='identity',  # 한글 소문자 변환 방지
    )
    
    print(f"✓ SentencePiece 토크나이저 학습 완료")
    
    sp = spm.SentencePieceProcessor()
    sp.Load("spm_korean.model")
    
    return sp


In [7]:
# ============================================================================
# 4. Positional Encoding
# ============================================================================

class PositionalEncoding(nn.Module):
    """위치 인코딩"""
    
    def __init__(self, position: int, d_model: int):
        super().__init__()
        self.d_model = d_model
        self.position = position
        
        # 위치 인코딩 계산
        pe = torch.zeros(position, d_model)
        pos = torch.arange(0, position, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * 
            -(math.log(10000.0) / d_model)
        )
        
        pe[:, 0::2] = torch.sin(pos * div_term)
        if d_model % 2 == 1:
            pe[:, 1::2] = torch.cos(pos * div_term[:-1])
        else:
            pe[:, 1::2] = torch.cos(pos * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (batch_size, seq_len, d_model)
        Returns:
            x + positional encoding
        """
        return x + self.pe[:, :x.size(1), :].to(x.device)



In [8]:
# ============================================================================
# 5. Attention 메커니즘
# ============================================================================

def scaled_dot_product_attention(
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    mask: torch.Tensor = None,
    dropout: nn.Module = None
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Scaled dot-product attention
    
    Args:
        query: (batch_size, heads, seq_len, depth)
        key: (batch_size, heads, seq_len, depth)
        value: (batch_size, heads, seq_len, depth)
        mask: attention mask
        dropout: dropout layer
    
    Returns:
        output, attention weights
    """
    depth = key.size(-1)
    
    # 스케일된 내적
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(depth)
    
    # 마스킹
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float('-inf'))
    
    # Softmax
    attention_weights = F.softmax(scores, dim=-1)
    
    # 드롭아웃
    if dropout is not None:
        attention_weights = dropout(attention_weights)
    
    # 값과의 가중합
    output = torch.matmul(attention_weights, value)
    
    return output, attention_weights


class MultiHeadAttention(nn.Module):
    """멀티헤드 어텐션"""
    
    def __init__(self, d_model: int, num_heads: int, dropout: float = 0.1):
        super().__init__()
        assert d_model % num_heads == 0, "d_model은 num_heads로 나누어떨어져야 함"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.depth = d_model // num_heads
        
        self.query_dense = nn.Linear(d_model, d_model)
        self.key_dense = nn.Linear(d_model, d_model)
        self.value_dense = nn.Linear(d_model, d_model)
        self.output_dense = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(
        self,
        query: torch.Tensor,
        key: torch.Tensor,
        value: torch.Tensor,
        mask: torch.Tensor = None
    ) -> torch.Tensor:
        """
        Args:
            query, key, value: (batch_size, seq_len, d_model)
            mask: attention mask
        
        Returns:
            output: (batch_size, seq_len, d_model)
        """
        batch_size = query.size(0)
        
        # Linear projection
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)
        
        # 멀티헤드로 변형
        query = query.view(batch_size, -1, self.num_heads, self.depth).transpose(1, 2)
        key = key.view(batch_size, -1, self.num_heads, self.depth).transpose(1, 2)
        value = value.view(batch_size, -1, self.num_heads, self.depth).transpose(1, 2)
        
        # Attention
        attn_output, _ = scaled_dot_product_attention(
            query, key, value, mask, self.dropout
        )
        
        # 헤드 결합
        attn_output = attn_output.transpose(1, 2).contiguous()
        attn_output = attn_output.view(batch_size, -1, self.d_model)
        
        # 최종 linear projection
        output = self.output_dense(attn_output)
        
        return output


In [9]:
# ============================================================================
# 6. 마스킹 함수
# ============================================================================

def create_padding_mask(x: torch.Tensor) -> torch.Tensor:
    """패딩 마스크 생성"""
    mask = (x == 0).float()
    # ✅ 입력과 같은 device에서 처리
    return mask.unsqueeze(1).unsqueeze(2).to(x.device)  # (batch, 1, 1, seq_len)


def create_look_ahead_mask(x: torch.Tensor) -> torch.Tensor:
    """Look-ahead 마스크 생성 (디코더용)"""
    seq_len = x.size(1)
    device = x.device  # ✅ 입력의 device 가져오기
    
    # ✅ 올바른 device에서 생성 (device 파라미터 지정)
    look_ahead_mask = 1 - torch.tril(torch.ones((seq_len, seq_len), device=device))
    
    # ✅ padding_mask도 같은 device에서 생성됨
    padding_mask = create_padding_mask(x)
    
    # ✅ 모두 같은 device에서 처리
    look_ahead_mask = look_ahead_mask.unsqueeze(0)
    combined_mask = torch.maximum(look_ahead_mask, padding_mask)
    
    return (1 - combined_mask).to(device)



In [10]:
# ============================================================================
# 7. 인코더/디코더 레이어
# ============================================================================

class EncoderLayer(nn.Module):
    """인코더 레이어"""
    
    def __init__(self, d_model: int, num_heads: int, units: int, dropout: float = 0.1):
        super().__init__()
        
        self.mha = MultiHeadAttention(d_model, num_heads, dropout)
        self.dropout1 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model, eps=1e-6)
        
        self.ffn = nn.Sequential(
            nn.Linear(d_model, units),
            nn.ReLU(),
            nn.Linear(units, d_model)
        )
        self.dropout2 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(d_model, eps=1e-6)
    
    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        # Self-attention
        attn_output = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output)
        out1 = self.norm1(x + attn_output)
        
        # Feed-forward
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        out2 = self.norm2(out1 + ffn_output)
        
        return out2


class DecoderLayer(nn.Module):
    """디코더 레이어"""
    
    def __init__(self, d_model: int, num_heads: int, units: int, dropout: float = 0.1):
        super().__init__()
        
        # Self-attention
        self.self_mha = MultiHeadAttention(d_model, num_heads, dropout)
        self.dropout1 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model, eps=1e-6)
        
        # Encoder-decoder attention
        self.encdec_mha = MultiHeadAttention(d_model, num_heads, dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(d_model, eps=1e-6)
        
        # Feed-forward
        self.ffn = nn.Sequential(
            nn.Linear(d_model, units),
            nn.ReLU(),
            nn.Linear(units, d_model)
        )
        self.dropout3 = nn.Dropout(dropout)
        self.norm3 = nn.LayerNorm(d_model, eps=1e-6)
    
    def forward(
        self,
        x: torch.Tensor,
        encoder_output: torch.Tensor,
        look_ahead_mask: torch.Tensor = None,
        padding_mask: torch.Tensor = None
    ) -> torch.Tensor:
        # Self-attention
        self_attn = self.self_mha(x, x, x, look_ahead_mask)
        self_attn = self.dropout1(self_attn)
        out1 = self.norm1(x + self_attn)
        
        # Encoder-decoder attention
        encdec_attn = self.encdec_mha(out1, encoder_output, encoder_output, padding_mask)
        encdec_attn = self.dropout2(encdec_attn)
        out2 = self.norm2(out1 + encdec_attn)
        
        # Feed-forward
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output)
        out3 = self.norm3(out2 + ffn_output)
        
        return out3


In [11]:
# ============================================================================
# 8. 인코더/디코더
# ============================================================================

class Encoder(nn.Module):
    """인코더"""
    
    def __init__(
        self,
        vocab_size: int,
        num_layers: int,
        units: int,
        d_model: int,
        num_heads: int,
        dropout: float = 0.1
    ):
        super().__init__()
        self.d_model = d_model
        
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(position=5000, d_model=d_model)
        
        self.enc_layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, units, dropout)
            for _ in range(num_layers)
        ])
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        # 임베딩 및 포지셔널 인코딩
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)
        
        # 인코더 레이어 통과
        for layer in self.enc_layers:
            x = layer(x, mask)
        
        return x


class Decoder(nn.Module):
    """디코더"""
    
    def __init__(
        self,
        vocab_size: int,
        num_layers: int,
        units: int,
        d_model: int,
        num_heads: int,
        dropout: float = 0.1
    ):
        super().__init__()
        self.d_model = d_model
        
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(position=5000, d_model=d_model)
        
        self.dec_layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, units, dropout)
            for _ in range(num_layers)
        ])
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(
        self,
        x: torch.Tensor,
        encoder_output: torch.Tensor,
        look_ahead_mask: torch.Tensor = None,
        padding_mask: torch.Tensor = None
    ) -> torch.Tensor:
        # 임베딩 및 포지셔널 인코딩
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)
        
        # 디코더 레이어 통과
        for layer in self.dec_layers:
            x = layer(x, encoder_output, look_ahead_mask, padding_mask)
        
        return x


In [12]:
# ============================================================================
# 9. 전체 Transformer 모델
# ============================================================================

class Transformer(nn.Module):
    """Transformer 모델"""
    
    def __init__(
        self,
        vocab_size: int,
        num_layers: int,
        units: int,
        d_model: int,
        num_heads: int,
        dropout: float = 0.1
    ):
        super().__init__()
        
        self.encoder = Encoder(vocab_size, num_layers, units, d_model, num_heads, dropout)
        self.decoder = Decoder(vocab_size, num_layers, units, d_model, num_heads, dropout)
        
        self.final_layer = nn.Linear(d_model, vocab_size)
    
    def forward(
        self,
        encoder_input: torch.Tensor,
        decoder_input: torch.Tensor
    ) -> torch.Tensor:
        # 마스킹
        encoder_mask = None
        
        decoder_look_ahead_mask = create_look_ahead_mask(decoder_input)
        decoder_padding_mask = None
        
        # 인코더
        encoder_output = self.encoder(encoder_input, encoder_mask)
        
        # 디코더
        decoder_output = self.decoder(
            decoder_input,
            encoder_output,
            decoder_look_ahead_mask,
            decoder_padding_mask
        )
        
        # 출력층
        logits = self.final_layer(decoder_output)
        
        return logits


In [15]:
# ============================================================================
# 10. 데이터셋
# ============================================================================

class ChatbotDataset(Dataset):
    """챗봇 데이터셋"""
    
    def __init__(
        self,
        pairs: List[Tuple[str, str]],
        tokenizer,
        max_length: int = 50
    ):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.data = []
        
        for q, a in pairs:
            # 토크나이징
            q_ids = self.tokenizer.EncodeAsIds(q)
            a_ids = self.tokenizer.EncodeAsIds(a)
            
            # 길이 조정
            q_ids = self._pad_or_truncate(q_ids, max_length)
            a_ids = self._pad_or_truncate(a_ids, max_length)
            
            # 디코더 입력: [BOS] + answer
            dec_input = [self.tokenizer.bos_id()] + a_ids[:-1]
            dec_input = self._pad_or_truncate(dec_input, max_length)
            
            # 디코더 레이블: answer + [EOS]
            dec_label = a_ids[:]
            dec_label = self._pad_or_truncate(dec_label, max_length)
            
            self.data.append((
                torch.tensor(q_ids, dtype=torch.long),
                torch.tensor(dec_input, dtype=torch.long),
                torch.tensor(dec_label, dtype=torch.long)
            ))
    
    def _pad_or_truncate(self, ids: List[int], max_len: int) -> List[int]:
        """패딩 또는 자르기"""
        if len(ids) >= max_len:
            return ids[:max_len]
        else:
            return ids + [self.tokenizer.pad_id()] * (max_len - len(ids))
    
    def __len__(self) -> int:
        return len(self.data)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        return self.data[idx]



In [16]:
# ============================================================================
# 11. 손실 함수 (라벨 스무딩 포함)
# ============================================================================

class LabelSmoothingLoss(nn.Module):
    """라벨 스무딩을 포함한 손실 함수"""
    
    def __init__(self, vocab_size: int, padding_idx: int = 0, smoothing: float = 0.0):
        super().__init__()
        self.vocab_size = vocab_size
        self.padding_idx = padding_idx
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing
        
        self.criterion = nn.KLDivLoss(reduction='none')
    
    def forward(self, logits: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        Args:
            logits: (batch_size, seq_len, vocab_size)
            target: (batch_size, seq_len)
        
        Returns:
            loss: scalar
        """
        # Log softmax
        log_probs = F.log_softmax(logits, dim=-1)
        
        # 타겟을 원-핫 인코딩
        with torch.no_grad():
            true_dist = torch.zeros_like(log_probs)
            true_dist.scatter_(-1, target.unsqueeze(-1), self.confidence)
            true_dist += self.smoothing / (self.vocab_size - 1)
            
            # 패딩 토큰 제거
            mask = (target != self.padding_idx).unsqueeze(-1).float()
            true_dist = true_dist * mask
        
        # KL Divergence
        loss = self.criterion(log_probs, true_dist).sum(dim=-1)
        
        # 평균 (패딩 제외)
        mask = (target != self.padding_idx).float()
        return (loss * mask).sum() / mask.sum().clamp(min=1)



In [17]:
# ============================================================================
# 12. 메트릭
# ============================================================================

def accuracy_function(logits: torch.Tensor, targets: torch.Tensor, pad_id: int = 0) -> float:
    """정확도 계산"""
    preds = logits.argmax(dim=-1)
    mask = (targets != pad_id).float()
    correct = (preds == targets).float() * mask
    accuracy = correct.sum() / mask.sum().clamp(min=1)
    return accuracy.item()


def perplexity_function(loss: float) -> float:
    """Perplexity 계산"""
    return np.exp(loss)



In [18]:
# ============================================================================
# 13. 학습 함수
# ============================================================================

def get_lr_lambda(d_model: int, warmup_steps: int = 4000):
    """학습률 스케줄"""
    def lr_lambda(step: int) -> float:
        step = float(step + 1)
        warmup_steps_f = float(warmup_steps)
        d_model_f = float(d_model)
        return (d_model_f ** -0.5) * min(
            step ** -0.5,
            step * (warmup_steps_f ** -1.5)
        )
    return lr_lambda


def train_step(
    model: nn.Module,
    batch: Tuple[torch.Tensor, torch.Tensor, torch.Tensor],
    optimizer: optim.Optimizer,
    criterion: nn.Module,
    device: torch.device,
    scaler: GradScaler = None
) -> Tuple[float, float]:
    """한 스텝 학습"""
    model.train()
    
    enc_input, dec_input, target = [x.to(device) for x in batch]
    
    optimizer.zero_grad()
    
    # 혼합 정밀도 학습
    if scaler:
        with autocast():
            logits = model(enc_input, dec_input)
            loss = criterion(logits.view(-1, logits.size(-1)).to(device), target.view(-1).to(device))
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
    else:
        logits = model(enc_input, dec_input)
        loss = criterion(logits.view(-1, logits.size(-1)).to(device), target.view(-1).to(device))
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
    
    acc = accuracy_function(logits, target, pad_id=0)
    
    return loss.item(), acc


def validate(
    model: nn.Module,
    dataloader: DataLoader,
    criterion: nn.Module,
    device: torch.device
) -> Tuple[float, float]:
    """검증"""
    model.eval()
    total_loss = 0
    total_acc = 0
    
    with torch.no_grad():
        for batch in dataloader:
            enc_input, dec_input, target = [x.to(device) for x in batch]
            
            logits = model(enc_input, dec_input)
            loss = criterion(logits.view(-1, logits.size(-1)).to(device), target.view(-1).to(device))
            
            total_loss += loss.item()
            total_acc += accuracy_function(logits, target, pad_id=0)
    
    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc / len(dataloader)
    
    return avg_loss, avg_acc


In [19]:
# ============================================================================
# 14. 추론 함수 (Beam Search)
# ============================================================================

def beam_search_decode(
    model: nn.Module,
    encoder_input: torch.Tensor,
    tokenizer,
    device: torch.device,
    beam_size: int = 5,
    max_length: int = 50,
    length_penalty: float = 0.6
) -> List[int]:
    """
    Beam search를 사용한 디코딩
    """
    model.eval()
    
    START_TOKEN = tokenizer.bos_id()
    END_TOKEN = tokenizer.eos_id()
    
    # 인코더 처리
    with torch.no_grad():
        encoder_output = model.encoder(encoder_input)
    
    # 초기 후보
    candidates = [(0.0, [START_TOKEN])]
    
    for step in range(max_length - 1):
        all_candidates = []
        
        for log_prob, sequence in candidates:
            # 디코더 입력
            decoder_input = torch.tensor([sequence], device=device)
            
            with torch.no_grad():
                decoder_output = model.decoder(
                    decoder_input,
                    encoder_output,
                    create_look_ahead_mask(decoder_input),
                    None
                )
                logits = decoder_output[0, -1, :]
                log_probs = F.log_softmax(logits, dim=-1)
            
            # Top-k 선택
            top_k_log_probs, top_k_indices = torch.topk(log_probs, min(beam_size * 2, log_probs.size(0)))
            
            # 후보 추가
            for token_log_prob, token_id in zip(top_k_log_probs, top_k_indices):
                token_id = token_id.item()
                new_log_prob = log_prob + token_log_prob.item()
                new_sequence = sequence + [token_id]
                
                # 길이 정규화
                normalized_log_prob = new_log_prob / (len(new_sequence) ** length_penalty)
                all_candidates.append((normalized_log_prob, new_sequence))
        
        # 상위 beam_size개 유지
        all_candidates.sort(key=lambda x: x[0], reverse=True)
        candidates = all_candidates[:beam_size]
        
        # 모든 후보가 END_TOKEN으로 끝나면 종료
        if all(seq[-1] == END_TOKEN for _, seq in candidates):
            break
    
    best_sequence = candidates[0][1]
    
    return best_sequence


In [20]:
# ============================================================================
# 15. 추론 함수 (온도 샘플링)
# ============================================================================

def top_k_sampling(logits: torch.Tensor, k: int = 40, temperature: float = 0.8) -> int:
    """Top-k 샘플링"""
    logits = logits / temperature
    top_k_logits, top_k_indices = torch.topk(logits, k)
    
    logits_filtered = torch.full_like(logits, float('-inf'))
    logits_filtered[top_k_indices] = top_k_logits
    
    probs = F.softmax(logits_filtered, dim=-1)
    token = torch.multinomial(probs, 1)
    
    return token.item()


def nucleus_sampling(logits: torch.Tensor, p: float = 0.9, temperature: float = 0.8) -> int:
    """Nucleus (Top-p) 샘플링"""
    logits = logits / temperature
    probs = F.softmax(logits, dim=-1)
    
    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    cumsum_probs = torch.cumsum(sorted_probs, dim=-1)
    
    sorted_indices_to_remove = cumsum_probs > p
    sorted_indices_to_remove[0] = False
    
    sorted_probs[sorted_indices_to_remove] = 0.0
    sorted_probs = sorted_probs / sorted_probs.sum()
    
    sampled_idx = torch.multinomial(sorted_probs, 1)
    token = sorted_indices[sampled_idx].item()
    
    return token


def generate_response(
    model: nn.Module,
    input_text: str,
    tokenizer,
    device: torch.device,
    config: Config,
    method: str = 'beam_search'
) -> str:
    """응답 생성"""
    
    model.eval()
    
    # 전처리
    input_text = KoreanPreprocessor.preprocess_sentence(input_text)
    
    # 토크나이징
    input_ids = [tokenizer.bos_id()] + tokenizer.EncodeAsIds(input_text) + [tokenizer.eos_id()]
    input_ids = input_ids[:config.MAX_SEQ_LENGTH]
    input_ids += [tokenizer.pad_id()] * (config.MAX_SEQ_LENGTH - len(input_ids))
    
    encoder_input = torch.tensor([input_ids], device=device)
    
    # 생성 방법 선택
    if method == 'beam_search':
        output_seq = beam_search_decode(
            model, encoder_input, tokenizer, device,
            beam_size=config.BEAM_SIZE,
            length_penalty=config.LENGTH_PENALTY
        )
    elif method == 'temperature':
        # 온도 샘플링
        START_TOKEN = tokenizer.bos_id()
        END_TOKEN = tokenizer.eos_id()
        
        output_seq = [START_TOKEN]
        
        with torch.no_grad():
            encoder_output = model.encoder(encoder_input)
            
            for _ in range(config.MAX_SEQ_LENGTH - 1):
                decoder_input = torch.tensor([output_seq], device=device)
                decoder_output = model.decoder(
                    decoder_input, encoder_output,
                    create_look_ahead_mask(decoder_input), None
                )
                logits = decoder_output[0, -1, :]
                
                next_token = top_k_sampling(
                    logits, k=config.TOP_K, temperature=config.TEMPERATURE
                )
                output_seq.append(next_token)
                
                if next_token == END_TOKEN:
                    break
    else:
        raise ValueError(f"Unknown method: {method}")
    
    # 디코딩
    response = tokenizer.decode(
        [t for t in output_seq if t < tokenizer.GetPieceSize()]
    )
    
    return response



In [ ]:
# ============================================================================
# 16. 메인 학습 루프
# ============================================================================

def main():
    """메인 함수"""
    
    print("=" * 70)
    print("한국어 Transformer 챗봇 - 완전 통합")
    print("=" * 70)
    
    # 시드 설정
    torch.manual_seed(Config.SEED)
    np.random.seed(Config.SEED)

    # ✅ Device 설정 확인 및 고정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    Config.DEVICE = device  # Config의 DEVICE도 업데이트
    
    print(f"\n✓ 사용 기기: {device}")
    if torch.cuda.is_available():
        print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
        print(f"✓ GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
    
    # 설정 저장 (실패해도 계속 진행)
    try:
        Config.save("config.json")
    except Exception as e:
        print(f"⚠ 설정 저장 실패 (계속 진행): {e}")
    
    # ========== 데이터 로드 ==========
    print("\n[1단계] 데이터 로드")
    print("-" * 70)
    
    pairs = KoreanPreprocessor.load_data_from_csv(
        Config.DATA_PATH,
        max_samples=Config.MAX_SAMPLES
    )
    
    if not pairs:
        print("❌ 데이터를 로드할 수 없습니다.")
        print(f"다음 경로에 파일을 놓아주세요: {Path(Config.DATA_PATH).absolute()}")
        return
    
    print(f"✓ {len(pairs)}개 Q&A 쌍 로드됨")
    
    # ========== 토크나이저 학습 ==========
    print("\n[2단계] SentencePiece 토크나이저 학습")
    print("-" * 70)
    
    if not Path("spm_korean.model").exists():
        tokenizer = train_tokenizer(pairs, Config)
    else:
        tokenizer = spm.SentencePieceProcessor()
        tokenizer.Load("spm_korean.model")
        print("✓ 기존 토크나이저 로드")
    
    # ========== 데이터셋 준비 ==========
    print("\n[3단계] 데이터셋 준비")
    print("-" * 70)
    
    dataset = ChatbotDataset(pairs, tokenizer, max_length=Config.MAX_SEQ_LENGTH)
    
    # 학습/검증 분할
    train_size = int(len(dataset) * Config.TRAIN_VALID_SPLIT)
    valid_size = len(dataset) - train_size
    train_dataset, valid_dataset = torch.utils.data.random_split(
        dataset, [train_size, valid_size]
    )
    
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=Config.BATCH_SIZE, shuffle=False)
    
    print(f"✓ 학습 샘플: {len(train_dataset)}, 검증 샘플: {len(valid_dataset)}")
    
    # ========== 모델 생성 ==========
    print("\n[4단계] 모델 생성")
    print("-" * 70)
    
    model = Transformer(
        vocab_size=tokenizer.GetPieceSize(),
        num_layers=Config.NUM_LAYERS,
        units=Config.UNITS,
        d_model=Config.D_MODEL,
        num_heads=Config.NUM_HEADS,
        dropout=Config.DROPOUT
    )
    
    model = model.to(Config.DEVICE)
    
    # 모델 파라미터 개수
    total_params = sum(p.numel() for p in model.parameters())
    print(f"✓ 모델 파라미터: {total_params:,}")
    
    # ========== 옵티마이저 및 손실 함수 ==========
    print("\n[5단계] 최적화 설정")
    print("-" * 70)
    
    optimizer = optim.AdamW(
        model.parameters(),
        lr=Config.LEARNING_RATE,
        betas=Config.ADAM_BETAS,
        weight_decay=Config.WEIGHT_DECAY
    )
    
    scheduler = lr_scheduler.LambdaLR(
        optimizer,
        lr_lambda=get_lr_lambda(Config.D_MODEL, Config.WARMUP_STEPS)
    )
    
    criterion = LabelSmoothingLoss(
        vocab_size=tokenizer.GetPieceSize(),
        padding_idx=tokenizer.pad_id(),
        smoothing=Config.LABEL_SMOOTHING
    )
    
    # ✅ 중요: criterion도 device로 이동
    criterion = criterion.to(Config.DEVICE)
    scaler = GradScaler() if Config.USE_MIXED_PRECISION else None
    
    print(f"✓ 옵티마이저: AdamW")
    print(f"✓ 손실 함수: Label Smoothing (smoothing={Config.LABEL_SMOOTHING})")
    print(f"✓ 혼합 정밀도: {'활성화' if scaler else '비활성화'}")
    
    # ========== 학습 ==========
    print("\n[6단계] 모델 학습 시작")
    print("-" * 70)
    
    best_val_loss = float('inf')
    patience_count = 0
    
    for epoch in range(Config.NUM_EPOCHS):
        print(f"\n[에폭 {epoch+1}/{Config.NUM_EPOCHS}]")
        
        # 학습
        train_loss = 0
        train_acc = 0
        
        for step, batch in enumerate(train_loader):
            loss, acc = train_step(model, batch, optimizer, criterion, Config.DEVICE, scaler)
            
            train_loss += loss
            train_acc += acc
            
            scheduler.step()
            
            if step % Config.LOG_INTERVAL == 0:
                avg_loss = train_loss / (step + 1)
                avg_acc = train_acc / (step + 1)
                ppl = perplexity_function(avg_loss)
                lr = optimizer.param_groups[0]['lr']
                
                print(f"  Step {step:4d}/{len(train_loader):4d} | "
                      f"Loss: {avg_loss:.4f} | Acc: {avg_acc:.4f} | "
                      f"PPL: {ppl:.2f} | LR: {lr:.2e}")
        
        train_loss = train_loss / len(train_loader)
        train_acc = train_acc / len(train_loader)
        train_ppl = perplexity_function(train_loss)
        
        # 검증
        val_loss, val_acc = validate(model, valid_loader, criterion, Config.DEVICE)
        val_ppl = perplexity_function(val_loss)
        
        print(f"  Train | Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | PPL: {train_ppl:.2f}")
        print(f"  Valid | Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | PPL: {val_ppl:.2f}")
        
        # 체크포인트 저장
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_count = 0
            
            checkpoint_path = Config.SAVE_DIR / f"best_model.pt"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss': val_loss,
                'config': Config.to_dict()
            }, checkpoint_path)
            print(f"  ✓ 최고 모델 저장! (Val Loss: {val_loss:.4f})")
        else:
            patience_count += 1
            print(f"  ⚠ Val Loss 개선 없음 ({patience_count}/{Config.EARLY_STOPPING_PATIENCE})")
            
            if patience_count >= Config.EARLY_STOPPING_PATIENCE:
                print(f"\n✓ 조기 종료 (에폭 {epoch+1})")
                break
    
    # ========== 최고 모델 로드 ==========
    print("\n[7단계] 최고 모델 로드")
    print("-" * 70)
    
    best_checkpoint = torch.load(Config.SAVE_DIR / "best_model.pt", map_location=Config.DEVICE)
    model.load_state_dict(best_checkpoint['model_state_dict'])
    print("✓ 최고 모델 로드됨")
    
    # ========== 테스트 ==========
    print("\n[8단계] 응답 생성 테스트")
    print("-" * 70)
    
    test_inputs = [
        "안녕하세요",
        "오늘 날씨 어때요",
        "감사합니다",
        "뭐 하고 있어요",
    ]
    
    for input_text in test_inputs:
        print(f"\n입력: {input_text}")
        
        # Beam Search
        response_beam = generate_response(
            model, input_text, tokenizer, Config.DEVICE, Config, method='beam_search'
        )
        print(f"응답(Beam): {response_beam}")
        
        # 온도 샘플링
        response_temp = generate_response(
            model, input_text, tokenizer, Config.DEVICE, Config, method='temperature'
        )
        print(f"응답(온도): {response_temp}")
    
    print("\n" + "=" * 70)
    print("✓ 학습 완료!")
    print("=" * 70)


if __name__ == "__main__":
    main()


한국어 Transformer 챗봇 - 완전 통합

✓ 사용 기기: cuda
✓ GPU: Tesla T4
✓ GPU 메모리: 15.7GB
✓ 설정 저장: config.json

[1단계] 데이터 로드
----------------------------------------------------------------------
✓ 데이터 컬럼: ['Q', 'A', 'label']
✓ 71630개 Q&A 쌍 로드됨
✓ 71630개 Q&A 쌍 로드됨

[2단계] SentencePiece 토크나이저 학습
----------------------------------------------------------------------
✓ 기존 토크나이저 로드

[3단계] 데이터셋 준비
----------------------------------------------------------------------
✓ 학습 샘플: 64467, 검증 샘플: 7163

[4단계] 모델 생성
----------------------------------------------------------------------
✓ 모델 파라미터: 117,677,888

[5단계] 최적화 설정
----------------------------------------------------------------------
✓ 옵티마이저: AdamW
✓ 손실 함수: Label Smoothing (smoothing=0.2)
✓ 혼합 정밀도: 활성화

[6단계] 모델 학습 시작
----------------------------------------------------------------------

[에폭 1/100]
  Step    0/4030 | Loss: 6.7975 | Acc: 0.0000 | PPL: 895.58 | LR: 1.01e-11
  Step  100/4030 | Loss: 6.8105 | Acc: 0.0000 | PPL: 907.31 | LR: 5.14e-10
  Step  20

### 1차 실험(데이터 추가 X), 2차 실험(질의 응답 코퍼스 추가)

#### 1차 실험 결과 (100 에폭):
 - 손실 함수가 내려가는 양상은 보였으나 추론 진행하였을 때 상식적인 답변을 받지 못하였음 (valid Loss : 5.4742 | Acc : 0.1709)
 - 학습에 필요한 데이터 양이 적다고 판단하여 코퍼스 추가해서 2차 실험 판단

#### 2차 실험 결과 (100 에폭):
 - 손실 함수가 꾸준히 내려가는 양상을 보이나 완료시간이 엄청 길어서 물리적인 시간이 많이 듦 ( 10 에폭 결과 > valid Loss : 5.0561 | Acc : 0.1931)
 - 확실히 데이터를 추가하니 같은 에폭을 돌렸을 때보다 성능이 높아 지는 것으로 판단

종합 회고 : 복잡한 모델을 학습시키고 성능개선하기 상당히 많은 제약사항 (컴퓨팅 성능, 모델의 성능, 학습 데이터의 양, 데이터 전처리)을 경험하였고 좀더 실험을 체계적으로 세우고 세부적인 수정 및 개선하기위한 방법에 익숙해져야겠다는 생각이 든다 
  

### 코퍼스 추가 작업

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
한국어 Q&A 코퍼스 통합 다운로더

지원하는 코퍼스:
1. ChatBot 데이터 (GitHub) - 자동 다운로드
2. KorQA (수동 다운로드 후 자동 처리)

"""

import pandas as pd
import urllib.request
import json
import os
import sys


class KoreanQACorpusDownloader:
    """한국어 Q&A 코퍼스 통합 다운로더"""
    
    def __init__(self):
        self.output_file = "korean_qa_data.csv"
        self.dfs = []
    
    def download_chatbot_data(self):
        """ChatBot 데이터 다운로드 (GitHub에서 자동)"""
        print("\n📥 Step 1: ChatBot 데이터 다운로드 (GitHub)")
        print("-" * 60)
        
        url = "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv"
        
        print(f"  🌐 {url}")
        
        try:
            df = pd.read_csv(url, encoding='utf-8')
            print(f"  ✅ 다운로드 완료! ({len(df):,} 샘플)")
            
            return df
        
        except Exception as e:
            print(f"  ⚠️ 다운로드 실패: {e}")
            print(f"     (인터넷 연결 확인)")
            return None
    
    def parse_local_korquad(self, json_file='korquad_v2.json'):
        """로컬 KorQA 파일 파싱"""
        print("\n📥 Step 2: KorQA 데이터 파싱 (로컬)")
        print("-" * 60)
        
        if not os.path.exists(json_file):
            print(f"  ⚠️ 파일 없음: {json_file}")
            print(f"     다운로드: https://korquad.github.io/")
            return None
        
        print(f"  📂 파일: {json_file}")
        
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            qa_pairs = []
            
            # KorQA 형식 파싱
            for article in data.get('data', []):
                for paragraph in article.get('paragraphs', []):
                    for qa in paragraph.get('qas', []):
                        question = qa.get('question', '')
                        answers = qa.get('answers', [])
                        
                        if question and answers:
                            answer = answers[0].get('text', '')
                            qa_pairs.append({
                                'Q': question,
                                'A': answer,
                                'label': 'korquad'
                            })
            
            df = pd.DataFrame(qa_pairs)
            print(f"  ✅ 파싱 완료! ({len(df):,} 샘플)")
            
            return df
        
        except Exception as e:
            print(f"  ❌ 파싱 실패: {e}")
            return None
    
    
    def merge_and_save(self, *dataframes):
        """데이터 병합 및 저장"""
        print("\n🔗 Step 4: 데이터 병합 및 저장")
        print("-" * 60)
        
        dfs = [df for df in dataframes if df is not None]
        
        if not dfs:
            print("  ❌ 병합할 데이터가 없습니다")
            return None
        
        # 병합
        merged = pd.concat(dfs, ignore_index=True)
        print(f"  병합: {len(merged):,} 샘플")
        
        # 정제
        # 1. 빈 값 제거
        merged = merged.dropna()
        print(f"  정제: {len(merged):,} 샘플 (빈 값 제거)")
        
        # 2. 길이 확인
        merged = merged[
            (merged['Q'].str.len() > 1) & 
            (merged['A'].str.len() > 1)
        ]
        print(f"  정제: {len(merged):,} 샘플 (길이 확인)")
        
        # 3. 중복 제거
        original_len = len(merged)
        merged = merged.drop_duplicates(subset=['Q', 'A'], keep='first')
        removed = original_len - len(merged)
        print(f"  정제: {len(merged):,} 샘플 ({removed:,} 중복 제거)")
        
        # 저장
        merged.to_csv(self.output_file, index=False, encoding='utf-8')
        print(f"  ✅ 저장: {self.output_file}")
        
        return merged
    
    def print_samples(self, df):
        """샘플 출력"""
        if df is None or len(df) == 0:
            return
        
        print("\n📊 샘플 데이터:")
        print("=" * 70)
        
        for i, row in df.head(5).iterrows():
            print(f"\n[{i+1}] ({row['label']})")
            print(f"  Q: {row['Q'][:60]}")
            
            answer = row['A']
            if len(answer) > 60:
                answer = answer[:60] + "..."
            
            print(f"  A: {answer}")
    
    def run(self):
        """전체 프로세스 실행"""
        print("\n" + "=" * 70)
        print("한국어 Q&A 코퍼스 통합 다운로더")
        print("=" * 70)
        
        dfs = []
        
        # 1. ChatBot 데이터 (자동)
        df_chatbot = self.download_chatbot_data()
        if df_chatbot is not None:
            dfs.append(df_chatbot)
        
        # 2. KorQA 데이터 (수동 다운로드 후)
        if os.path.exists('korquad_v2.json'):
            df_korquad = self.parse_local_korquad('korquad_v2.json')
            if df_korquad is not None:
                dfs.append(df_korquad)
        else:
            print("\n⚠️  KorQA 파일 없음 (선택사항)")
            print("   다운로드: https://korquad.github.io/")
            print("   압축 해제 후 korquad_v2.json을 현재 디렉토리에 배치")
        
        # 병합
        if not dfs:
            print("\n❌ 처리할 데이터가 없습니다")
            print("   최소한 ChatBot 데이터는 자동으로 다운로드됩니다")
            print("   인터넷 연결을 확인하고 다시 시도하세요")
            return False
        
        merged = self.merge_and_save(*dfs)
        
        if merged is None or len(merged) == 0:
            print("\n❌ 병합 실패!")
            return False
        
        # 샘플 출력
        self.print_samples(merged)
        
        # 완료
        print("\n" + "=" * 70)
        print("✨ 완료!")
        print("=" * 70)
        
        # 통계
        file_size = os.path.getsize(self.output_file) / 1024 / 1024
        
        print(f"\n📈 최종 통계:")
        print(f"  파일: {self.output_file}")
        print(f"  샘플: {len(merged):,} 개")
        print(f"  크기: {file_size:.1f}MB")
        print(f"  Q 평균 길이: {merged['Q'].str.len().mean():.1f} 자")
        print(f"  A 평균 길이: {merged['A'].str.len().mean():.1f} 자")
        
        # 다음 단계
        print(f"\n🚀 다음 단계:")
        print(f"  1. merge_datasets.py 실행:")
        print(f"     $ python3 merge_datasets.py")
        print(f"")
        print(f"  2. Config 수정:")
        print(f"     DATA_PATH = 'korean_qa_data.csv'")
        print(f"")
        print(f"  3. 모델 재학습:")
        print(f"     $ python3 korean_chatbot_complete.py")
        
        print("\n" + "=" * 70)
        print("경고: merge_datasets.py를 사용할 때")
        print("      korean_qa_data.csv를 ChatbotData.csv와 병합할 수 있습니다")
        print("=" * 70 + "\n")
        
        return True


def main():
    """메인 함수"""
    downloader = KoreanQACorpusDownloader()
    success = downloader.run()
    
    return 0 if success else 1


if __name__ == "__main__":
    try:
        exit_code = main()
        sys.exit(exit_code)
    
    except KeyboardInterrupt:
        print("\n\n❌ 사용자에 의해 중단되었습니다")
        sys.exit(1)
    
    except Exception as e:
        print(f"\n❌ 예상치 못한 오류: {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)


한국어 Q&A 코퍼스 통합 다운로더

📥 Step 1: ChatBot 데이터 다운로드 (GitHub)
------------------------------------------------------------
  🌐 https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv
  ✅ 다운로드 완료! (11,823 샘플)

📥 Step 2: KorQA 데이터 파싱 (로컬)
------------------------------------------------------------
  📂 파일: korquad_v2.json
  ✅ 파싱 완료! (60,407 샘플)

⚠️  AI Hub 파일 없음 (선택사항)
   다운로드: https://aihub.or.kr/
   "한국어 대화" 검색 후 JSON 형식 다운로드
   파일명을 aihub_data.json으로 변경하여 현재 디렉토리에 배치

🔗 Step 4: 데이터 병합 및 저장
------------------------------------------------------------
  병합: 72,230 샘플
  정제: 72,230 샘플 (빈 값 제거)
  정제: 71,854 샘플 (길이 확인)
  정제: 71,630 샘플 (224 중복 제거)
  ✅ 저장: korean_qa_data.csv

📊 샘플 데이터:

[1] (0)
  Q: 12시 땡!
  A: 하루가 또 가네요.

[2] (0)
  Q: 1지망 학교 떨어졌어
  A: 위로해 드립니다.

[3] (0)
  Q: 3박4일 놀러가고 싶다
  A: 여행은 언제나 좋죠.

[4] (0)
  Q: 3박4일 정도 놀러가고 싶다
  A: 여행은 언제나 좋죠.

[5] (0)
  Q: PPL 심하네
  A: 눈살이 찌푸려지죠.

✨ 완료!

📈 최종 통계:
  파일: korean_qa_data.csv
  샘플: 71,630 개
  크기: 6.9MB
  Q 평균 길이: 30.4 자
 

SystemExit: 0

/opt/conda/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3680: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
